# Cas d'utilisation des valeurs et vecteurs propres

In [10]:
import numpy as np
import numpy.linalg as lin

np.set_printoptions(precision=3, linewidth=150, suppress=True)

## Fibonacci

La suite de Fibonacci est définie par $x_n = x_{n-2} + x_{n-1}$ avec $x_0 = 1$ et $x_1 = 1$.

On vera des algorithmes pour calculer $x_n$.


Comme on a 2 termes à droite du signe égal, écrivons Fibonnacci sous forme d'un système matriciel $2\times 2$ :

$
\begin{align}
x_{n-1} &= x_{n-1} \\
x_n &= x_{n-2} + x_{n-1} \\
\end{align}
$
ce qui donne
$
\begin{bmatrix}
x_{n-1}\\
x_n  \\
\end{bmatrix}
=
\begin{bmatrix}
0 & 1 \\
1 & 1 \\
\end{bmatrix}
\begin{bmatrix}
x_{n-2}\\
x_{n-1}  \\
\end{bmatrix}
$

Écrire la matrice F ci-dessous.

In [11]:
F = np.array([[0,1], [1,1]])

Calculer les 6 premiers éléments de notre « suite vectorielle » de Fibonacci, en n'utilisant que la matrice F et les valeurs initiales de la suite. La fonction `numpy.linalg.matrix_power` pourra vous être utile.

In [12]:
x0 = np.array([1,1])
def fibo(n):
    x = x0
    for i in range(n):
        x = F@x
        print(x)
    return x
#print(fibo(4))
fibo(4)

[1 2]
[2 3]
[3 5]
[5 8]


array([5, 8])

In [13]:
x0 = np.array([1,1])
for i in range(6):
    print(lin.matrix_power(F, i) @ x0)


[1 1]
[1 2]
[2 3]
[3 5]
[5 8]
[ 8 13]


Calculer n produits matriciels n'est pas rentable, par contre sachant que
$F = P\, D\, P^{-1}$ avec

* P la matrice des vecteurs propres
* D la matrice diagonale des valeurs propres

on peut faire quelque chose car il va avoir des simplification lors du calcul de de $F^n$. 

* Ecrire la formule matriciel de la suite de Fibonnacci en fonction de P et D. 
* Expliquer pourquoi le calcul du n-ème élément de la suite peut se faire en temps constant.

$
\begin{bmatrix}
x_{n}\\
x_{n+1}  \\
\end{bmatrix}
=
\begin{bmatrix}
0 & 1 \\
1 & 1 \\
\end{bmatrix}^n
\begin{bmatrix}
x_{0}\\
x_{1}  \\
\end{bmatrix}
= (P\, D\, P^{-1})^n
\begin{bmatrix}
x_{0}\\
x_{1}  \\
\end{bmatrix}
= P\, D^n\, P^{-1}
\begin{bmatrix}
x_{0}\\
x_{1}  \\
\end{bmatrix}
$

On peut donc calculer $x_n$ en temps constant  puisque $D^n$ se calcule en temps constant, (D étant diagonale, $D^n$ et la puissance n de chaque valeur de la diagonale).

Écrire une fonction `fibo(n)` qui calcule le n-ème élément de la suite de Fibonacci en temps constant.

In [14]:
fval, fvec = lin.eig(F)
fval = fval.astype('float')  # la matrice est symétrique donc ses valeurs propres sont réelles
print("Valeurs propres de la matrice de fibonnacci :", fval,"\n")
print("Vecteurs propres de la matrice de fibonnacci :\n", fvec)

Valeurs propres de la matrice de fibonnacci : [-0.618  1.618] 

Vecteurs propres de la matrice de fibonnacci :
 [[-0.851 -0.526]
 [ 0.526 -0.851]]


In [15]:
def fibo(n):
    return (fvec @ np.diag(fval**n) @ lin.inv(fvec) @ x0)[0]

fibo(50) # on a une erreur de 10⁻⁵, ca va

20365011074.00003

Vérifier que votre fonction est en temps constant en chronométrant `fibo(5)` et `fibo(1000)` avec la commande 
`%timeit` de Jupyter.

In [7]:
%timeit fibo(5)

6.86 µs ± 180 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [8]:
%timeit fibo(1000)

6.96 µs ± 98.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Google page rank

Soit N pages web numérotées qui font référence les unes aux autres. Disons que si la page 3 est référencée par les pages 8 9 et 13 j'écris $x_3 = x_8 + x_9 + x_{13}$. On voit qu'on peut écrire ces référencements dans une
matrice avec le i-ième ligne qui décrit par qui est référencée la i-ème page web. Cette matrice a un 1 dans
la j-ième colonne si la page j cite la page i et sinon un 0.

In [16]:
np.random.seed(42)
N = 8
A = np.random.randint(2,size=(N,N))
for i in range(len(A)):
    A[i,i] = 0   # on ne compte pas les auto-référencements
A

array([[0, 1, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0],
       [1, 1, 0, 0, 1, 0, 1, 1],
       [1, 1, 1, 0, 1, 1, 0, 0],
       [1, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 0, 1, 0],
       [1, 1, 0, 1, 0, 1, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0]])

On va utiliser cette matrice pour classer nos pages web selon leur importance. On part du principe qu'une page importante est référenciée par beaucoup d'autres pages importantes. (Inversement, personne a envie de faire des liens vers une page sans importance.)

Cette idée est derrière l'algorithme PageRank que Google utilisait (et peut-être toujours utilise) pour ranger les résultats d'une recherche internet.

## Approche itérative

Dans un premier temps on procède de la manière suivante pour trouver l'importance d'une page:
Étant donné N pages à évaluer 
* On initialise un vecteur __v__ de taille N avec 1 partout -> Correspond à une distribution uniforme de l'importance initialement
* On calcule le produit A __v__
* On normalise le résultat pour que la norme 1 soit égale au nombre de pages (N)
* Cela nous donne l'importance de chaque page selon le nombre de liens vers elle des autres pages
* On répète ces étapes jusqu'à un point fixe

De cette manière l'entrée i du vecteur __v__ correspond à l'importance de la page i. *Discutez pourquoi.*

In [22]:
v = np.zeros((N,1))
vprime = np.ones((N,1))

while (np.absolute(np.max(vprime - v)) > 1e-5): # which norm is used here?
    v = vprime
    vprime =  A @ v
    vprime = vprime * N / vprime.sum()   # all values are positive
    print(vprime.T)


[[0.615 0.308 1.538 1.538 0.923 1.231 1.538 0.308]]
[[0.513 0.513 1.231 1.538 0.821 1.846 1.333 0.205]]
[[0.776 0.439 1.114 1.62  0.743 1.62  1.519 0.169]]
[[0.668 0.493 1.184 1.523 0.756 1.622 1.501 0.252]]
[[0.689 0.489 1.196 1.539 0.764 1.618 1.486 0.218]]
[[0.685 0.483 1.186 1.547 0.772 1.621 1.481 0.224]]
[[0.685 0.482 1.187 1.546 0.767 1.624 1.485 0.223]]
[[0.686 0.484 1.186 1.545 0.767 1.624 1.485 0.223]]
[[0.686 0.484 1.187 1.545 0.767 1.623 1.485 0.223]]
[[0.686 0.484 1.187 1.545 0.767 1.623 1.485 0.223]]
[[0.686 0.484 1.187 1.545 0.767 1.623 1.485 0.223]]
[[0.686 0.484 1.187 1.545 0.767 1.623 1.485 0.223]]
[[0.686 0.484 1.187 1.545 0.767 1.623 1.485 0.223]]
[[0.686 0.484 1.187 1.545 0.767 1.623 1.485 0.223]]


## Un autre approche

On a calculé un vecteur __v__ pour lequel A.__v__ = __v__. Est-ce que ça vous rappelle quelque chose ?

* On considère que les composantes du premier vecteur propre représentent la valeur de chaque page. Calculez les valeurs des pages. Comparez avec le résultat obtenue auparavant.
* Calculez le nombre de citations pour chaque page.

In [24]:
val_pr, vec_pr = lin.eig(A)
#print(vec_pr)
#print(vec_pr[0])
#print(vec_pr[:,0])
order = np.argsort(np.absolute(val_pr))[::-1]    # on range les valeurs propres dans l'ordre décroissant de leur module
val_pr, vec_pr = val_pr[order], vec_pr[:, order] # ce sont les colonnes des vecteurs propres qu'on réordonne
print("Valeurs propres de la matrice des liens :", val_pr,"\n")
print("Vecteurs propres de la matrice des liens :\n", vec_pr)

Valeurs propres de la matrice des liens : [ 3.071+0.j     0.288-1.202j  0.288+1.202j -0.62 -0.948j -0.62 +0.948j -1.108+0.j    -1.   +0.j    -0.3  +0.j   ] 

Vecteurs propres de la matrice des liens :
 [[-0.217+0.j     0.021-0.168j  0.021+0.168j -0.481-0.j    -0.481+0.j    -0.368+0.j    -0.   +0.j     0.147+0.j   ]
 [-0.153+0.j     0.277-0.074j  0.277+0.074j  0.054+0.204j  0.054-0.204j  0.193+0.j     0.   +0.j     0.289+0.j   ]
 [-0.376+0.j     0.228+0.326j  0.228-0.326j  0.437+0.048j  0.437-0.048j -0.493+0.j     0.707+0.j    -0.525+0.j   ]
 [-0.489+0.j    -0.389+0.165j -0.389-0.165j -0.318-0.142j -0.318+0.142j -0.135+0.j    -0.   +0.j     0.413+0.j   ]
 [-0.243+0.j     0.033+0.43j   0.033-0.43j  -0.191-0.114j -0.191+0.114j  0.603+0.j    -0.707+0.j     0.299+0.j   ]
 [-0.514+0.j    -0.472-0.j    -0.472+0.j     0.244+0.252j  0.244-0.252j  0.215+0.j     0.   +0.j    -0.333+0.j   ]
 [-0.47 +0.j    -0.009-0.354j -0.009+0.354j  0.16 -0.178j  0.16 +0.178j -0.214+0.j    -0.   +0.j    -0.087+0

In [14]:
valeurs = np.abs(vec_pr[:,0]).astype(float)
print("Valeur des pages                 :", valeurs)
print("Valeur des pages normalisé à N   :", N * valeurs / valeurs.sum())
print("Nombre de citations              :", A.sum(axis=1))

Valeur des pages                 : [0.217 0.153 0.376 0.489 0.243 0.514 0.47  0.071]
Valeur des pages normalisé à N   : [0.686 0.484 1.187 1.545 0.767 1.623 1.485 0.223]
Nombre de citations              : [2 1 5 5 3 4 5 1]
